<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
</p>

# Python, Jupyter & pandas tutorial: Module 3

## Viewing geolocated data

In Module 2, we retrieved NSIDC snow-cover data from an OpenDAP server. Let's pull that data again for use in this notebook:

In [ ]:
import netCDF4
import numpy as np
url = ('http://opendap.apps.nsidc.org:80/opendap/DATASETS/'
       'nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc?'
       'latitude[0:1:719][0:1:719],'
       'longitude[0:1:719][0:1:719],'
       'merged_snow_cover_extent[0:1:0][0:1:719][0:1:719]'
)
dataset = netCDF4.Dataset(url)
latitude = np.array(dataset.variables['latitude'])
longitude = np.array(dataset.variables['longitude'])
msce = np.array(dataset.variables['merged_snow_cover_extent'])[0, :, :]

And now let's pull in some graphics support now and get ready to plot our snow-cover data.

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

It will be helpful to have an underlying map image to better interpret our data, so let's be sure we can at least do that:

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='npstere', boundinglat=45, lon_0=300)
map.drawcoastlines()
plt.draw()

We can ask for more interesting background imagery -- though, in general, we won't in order to focus on the data.

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='npstere', boundinglat=45, lon_0=300)
map.drawcoastlines()
map.etopo()
plt.draw()

Let's take a quick look back at the metadata for our Merged Snow Cover Extent variable:

In [ ]:
dataset.variables['merged_snow_cover_extent']

So, the snow-cover variable's values consist of discrete integers whose values represent snow cover from different sources, snow-free land, permanent ice-covered land, ocean, or a fill value.

As a reality check, let's find the would-be ocean points and plot them on our map:

# Yo! Is there a nicer way to construct the lats / lons lists (below) with NumPy, list comprehensions, etc.? Or should we do it the 'dumb' way here and show nicer filtering with pandas?

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='npstere', boundinglat=45, lon_0=300)
map.drawcoastlines()
latitude = np.array(latitude)
longitude = np.array(longitude)
lats = []
lons = []
for row in range(0, latitude.shape[0]):
    for col in range(0, latitude.shape[1]):
        lat = latitude[row][col]
        lon = longitude[row][col]
        cover = msce[row][col]
        if lat >= 45 and cover == 40:
            lats.append(lat)
            lons.append(lon)
x, y = map(lons, lats)
map.plot(x, y, '.', color='SeaGreen')
plt.draw()

Seems legit.

Now, where's the snow? Values 10 through 16 in the merged snow-cover variable all indicate snow, so we'll accept anything in that range:

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='npstere', boundinglat=45, lon_0=300)
map.drawcoastlines()
latitude = np.array(latitude)
longitude = np.array(longitude)
lats = []
lons = []
for row in range(0, latitude.shape[0]):
    for col in range(0, latitude.shape[1]):
        lat = latitude[row][col]
        lon = longitude[row][col]
        cover = msce[row][col]
        if lat >= 45 and cover in range(10, 17):
            lats.append(lat)
            lons.append(lon)
x, y = map(lons, lats)
map.plot(x, y, '.', color='Cyan')
plt.draw()

How about snow **or** permanent ice covered land?

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='npstere', boundinglat=45, lon_0=300)
map.drawcoastlines()
latitude = np.array(latitude)
longitude = np.array(longitude)
snow_lats = []
snow_lons = []
ice_lats = []
ice_lons = []
for row in range(0, latitude.shape[0]):
    for col in range(0, latitude.shape[1]):
        lat = latitude[row][col]
        lon = longitude[row][col]
        cover = msce[row][col]
        if lat >= 45:
            if cover in range(10, 17):
                snow_lats.append(lat)
                snow_lons.append(lon)
            elif cover == 30:
                ice_lats.append(lat)
                ice_lons.append(lon)
snow_x, snow_y = map(snow_lons, snow_lats)
map.plot(snow_x, snow_y, '.', color='Cyan')
ice_x, ice_y = map(ice_lons, ice_lats)
map.plot(ice_x, ice_y, '.', color='Blue')
plt.draw()

At the end of Module 2, we tried to use OpenDAP to retrieve data only in the area of Iceland. Let's execute that query again:

In [ ]:
iceland_url = ('http://opendap.apps.nsidc.org:80/opendap/DATASETS/'
       'nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc?'
       'latitude[453:1:476][310:1:338],'
       'longitude[453:1:476][310:1:338],'
       'merged_snow_cover_extent[0:1:0][453:1:476][310:1:338]'
)
iceland_dataset = netCDF4.Dataset(iceland_url)
iceland_latitude = np.array(iceland_dataset.variables['latitude'])
iceland_longitude = np.array(iceland_dataset.variables['longitude'])
iceland_msce = np.array(iceland_dataset.variables['merged_snow_cover_extent'])[0, :, :]

And now let's plot it on a map and see if the result makes sense:

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='npstere', boundinglat=45, lon_0=300)
map.drawcoastlines()
snow_lats = []
snow_lons = []
ice_lats = []
ice_lons = []
for row in range(0, iceland_latitude.shape[0]):
    for col in range(0, iceland_latitude.shape[1]):
        lat = iceland_latitude[row][col]
        lon = iceland_longitude[row][col]
        cover = iceland_msce[row][col]
        if lat >= 45:
            if cover in range(10, 17):
                snow_lats.append(lat)
                snow_lons.append(lon)
            elif cover == 30:
                ice_lats.append(lat)
                ice_lons.append(lon)
snow_x, snow_y = map(snow_lons, snow_lats)
map.plot(snow_x, snow_y, '.', color='Cyan')
ice_x, ice_y = map(ice_lons, ice_lats)
map.plot(ice_x, ice_y, '.', color='Blue')
plt.draw()

Looks about right, but let's get a closer look. Can we plot a map of just Iceland?

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67)
map.drawcoastlines()
plt.draw()

Well, that's pretty awful, and doesn't look much like Iceland:

<img src='images/iceland.png' style='height:459px;width:565px;float:left'/>

But we can do better, by replacing matplotlib's internal maps with a vector-based shapefile from the [Natural Earth](http://www.naturalearthdata.com/) project. In this case, we'll use Coastlines shapefile from the [1:50m Physical Vectors](http://www.naturalearthdata.com/downloads/50m-physical-vectors/) product. (There are finer 1:10m coastlines, too.)

In [ ]:
notebook_dir = %env PWD

In [ ]:
%%bash -es $notebook_dir
mkdir -p $1/iceland
cd $1/iceland
wget http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/physical/ne_50m_coastline.zip \
  -O ne_50m_coastline.zip
unzip ne_50m_coastline.zip

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67)
map.readshapefile('%s/iceland/ne_50m_coastline' % notebook_dir, 'iceland')
plt.draw()

That's much better.

Let's plot the snow- and ice-cover points. We can re-use the `[snow|ice]_[lats|lons]` variables we computed previously, but need to recompute the coordinates since we are using a different map projection (Mercator, instead of North Polar Stereographic). Let's also make the points bigger to fill in the space better.

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67)
map.readshapefile('%s/iceland/ne_50m_coastline' % notebook_dir, 'iceland')
snow_x, snow_y = map(snow_lons, snow_lats)
map.plot(snow_x, snow_y, '.', color='Cyan', markersize='24')
ice_x, ice_y = map(ice_lons, ice_lats)
map.plot(ice_x, ice_y, '.', color='Blue', markersize='24')
plt.draw()

If we add some (admittedly lo-res) background imagery, we can see that the permanent ice is located in what appear to be sensible places:

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
map = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67)
map.readshapefile('%s/iceland/ne_50m_coastline' % notebook_dir, 'iceland')
map.plot(snow_x, snow_y, '.', color='Cyan', markersize='24')
map.plot(ice_x, ice_y, '.', color='Blue', markersize='24')
map.bluemarble()
plt.draw()

In Module 4, we'll look at using pandas to do some data analysis.